---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [39]:
import pandas as pd
import numpy as np
from scipy import stats
#from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
import pandas as pd
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan","Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State","RegionName"]  )'''
    
    towns = pd.read_table('university_towns.txt',
                           names = ['head1'],
                           sep = '\n')
     
    states = towns[towns['head1'].str.contains("edit")]
    states['State'] = states['head1'].str[:-6]
    states = states.drop_duplicates(subset='State')
    
    towns_states = pd.merge(towns,states,on='head1', how = 'outer')
    
    towns_states['State'] = towns_states['State'].ffill()
    
    towns_states = towns_states[~towns_states['head1'].str.contains("edit")]
    
    towns_states['RegionName'] = towns_states['head1'].str.replace(r'[\(\[].*?[\)\]]', '')
    
    towns_states = towns_states.drop('head1', 1)
    
    return towns_states

get_list_of_university_towns()

/Users/Richard/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
6,Alabama,Troy
7,Alabama,Tuscaloosa
8,Alabama,Tuskegee
10,Alaska,Fairbanks
12,Arizona,Flagstaff


In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    pd.set_option('display.max_rows', 50)
    
    gdp = pd.read_excel('gdplev.xls',
                         skiprows = 8,
                         #skip_footer = 38,
                         parse_cols = "E:F",header = None)
    
    gdp = gdp.dropna(axis=0)
    
    gdp.columns = ['YYYYQQ', 'GDP_Curr']
    
    gdp['Year'] = gdp['YYYYQQ'].str[:4]
    gdp['Year'] = gdp['Year'].astype(int)
    
    gdp = gdp[gdp['Year'] >= 2000]
    
    gdp['gdp_lag'] = gdp['GDP_Curr'].shift(1)
    gdp['diff'] = gdp['GDP_Curr'] - gdp['gdp_lag']
    gdp['diff_lag'] = gdp['diff'].shift(1)
    
    recession_start = gdp[(gdp['diff'] < 0) & (gdp['diff_lag'] < 0)]

    recession_start = recession_start[:1]
    
    answer = recession_start['YYYYQQ']
    answer = answer.to_string()
    answer = answer[-6:]
                                    
    return answer
get_recession_start()

'2009q1'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    recession_start = get_recession_start()
    
    pd.set_option('display.max_rows', 500)
    
    gdp = pd.read_excel('gdplev.xls',
                         skiprows = 8,
                         #skip_footer = 38,
                         parse_cols = "E:F",header = None)
    
    gdp = gdp.dropna(axis=0)
    
    gdp.columns = ['YYYYQQ', 'GDP_Curr']
    
    gdp['Year'] = gdp['YYYYQQ'].str[:4]
    gdp['Year'] = gdp['Year'].astype(int)
    
    gdp = gdp[gdp['Year'] >= 2000]
    
    recession_st_row = gdp[gdp['YYYYQQ'] == recession_start].index[0]
    
    gdp_recession = gdp.loc[recession_st_row:]

    gdp_recession['gdp_lag'] = gdp_recession['GDP_Curr'].shift(1)
    gdp_recession['diff'] = gdp_recession['GDP_Curr'] - gdp_recession['gdp_lag']
    gdp_recession['diff_lag'] = gdp_recession['diff'].shift(1)
    
    recession_end = gdp_recession[(gdp_recession['diff'] > 0) & (gdp_recession['diff_lag'] > 0)]
    
    recession_end = recession_end[:1]
    
    answer = recession_end['YYYYQQ']
    answer = answer.to_string()
    answer = answer[-6:]

    return answer

get_recession_end()

/Users/Richard/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Richard/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Richard/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    recession_start = get_recession_start()
    recession_end = get_recession_end()
    
    pd.set_option('display.max_rows', 50)
    
    gdp = pd.read_excel('gdplev.xls',
                         skiprows = 8,
                         #skip_footer = 38,
                         parse_cols = "E:F",header = None)
    
    gdp = gdp.dropna(axis=0)
    
    gdp.columns = ['YYYYQQ', 'GDP_Curr']
    
    gdp['Year'] = gdp['YYYYQQ'].str[:4]
    gdp['Year'] = gdp['Year'].astype(int)
    
    gdp = gdp[gdp['Year'] >= 2000]
    
    recession_st_row = gdp[gdp['YYYYQQ'] == recession_start].index[0]
    recession_end_row = gdp[gdp['YYYYQQ'] == recession_end].index[0]
    
    gdp_recession = gdp.loc[recession_st_row:recession_end_row]

    recession_bottom = gdp_recession.loc[gdp_recession['GDP_Curr'].idxmin()]
    
    answer = recession_bottom['YYYYQQ']
    
    return answer
get_recession_bottom()

/Users/Richard/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Richard/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Richard/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

'2009q2'

In [70]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    housing_data = pd.read_csv("City_Zhvi_AllHomes.csv")
    housing_data['State'] = housing_data['State'].map(states)

    housing_data = housing_data.set_index(['State', 'RegionName'])
    
    housing_data = housing_data.ix[:,'2000-01':'2016-09']
    
    # 2000
    housing_data['2000q1'] = housing_data.ix[:,0:3].mean(axis=1)
    housing_data['2000q2'] = housing_data.ix[:,3:6].mean(axis=1)
    housing_data['2000q3'] = housing_data.ix[:,6:9].mean(axis=1)
    housing_data['2000q4'] = housing_data.ix[:,9:12].mean(axis=1)

    # 2001
    housing_data['2001q1'] = housing_data.ix[:,12:15].mean(axis=1)
    housing_data['2001q2'] = housing_data.ix[:,15:18].mean(axis=1)
    housing_data['2001q3'] = housing_data.ix[:,18:21].mean(axis=1)
    housing_data['2001q4'] = housing_data.ix[:,21:24].mean(axis=1)
    
    # 2002
    housing_data['2002q1'] = housing_data.ix[:,24:27].mean(axis=1)
    housing_data['2002q2'] = housing_data.ix[:,27:30].mean(axis=1)
    housing_data['2002q3'] = housing_data.ix[:,30:33].mean(axis=1)
    housing_data['2002q4'] = housing_data.ix[:,33:36].mean(axis=1)
    
    # 2003
    housing_data['2003q1'] = housing_data.ix[:,36:39].mean(axis=1)
    housing_data['2003q2'] = housing_data.ix[:,39:42].mean(axis=1)
    housing_data['2003q3'] = housing_data.ix[:,42:45].mean(axis=1)
    housing_data['2003q4'] = housing_data.ix[:,45:48].mean(axis=1)

    # 2004
    housing_data['2004q1'] = housing_data.ix[:,48:51].mean(axis=1)
    housing_data['2004q2'] = housing_data.ix[:,51:54].mean(axis=1)
    housing_data['2004q3'] = housing_data.ix[:,54:57].mean(axis=1)
    housing_data['2004q4'] = housing_data.ix[:,57:60].mean(axis=1)
    
    # 2005
    housing_data['2005q1'] = housing_data.ix[:,60:63].mean(axis=1)
    housing_data['2005q2'] = housing_data.ix[:,63:66].mean(axis=1)
    housing_data['2005q3'] = housing_data.ix[:,66:69].mean(axis=1)
    housing_data['2005q4'] = housing_data.ix[:,69:72].mean(axis=1)
    
    # 2006
    housing_data['2006q1'] = housing_data.ix[:,72:75].mean(axis=1)
    housing_data['2006q2'] = housing_data.ix[:,75:78].mean(axis=1)
    housing_data['2006q3'] = housing_data.ix[:,78:81].mean(axis=1)
    housing_data['2006q4'] = housing_data.ix[:,81:84].mean(axis=1)
    
    # 2007
    housing_data['2007q1'] = housing_data.ix[:,84:87].mean(axis=1)
    housing_data['2007q2'] = housing_data.ix[:,87:90].mean(axis=1)
    housing_data['2007q3'] = housing_data.ix[:,90:93].mean(axis=1)
    housing_data['2007q4'] = housing_data.ix[:,93:96].mean(axis=1)
    
    # 2008
    housing_data['2008q1'] = housing_data.ix[:,96:99].mean(axis=1)
    housing_data['2008q2'] = housing_data.ix[:,99:102].mean(axis=1)
    housing_data['2008q3'] = housing_data.ix[:,102:105].mean(axis=1)
    housing_data['2008q4'] = housing_data.ix[:,105:108].mean(axis=1)
    
    # 2009
    housing_data['2009q1'] = housing_data.ix[:,108:111].mean(axis=1)
    housing_data['2009q2'] = housing_data.ix[:,111:114].mean(axis=1)
    housing_data['2009q3'] = housing_data.ix[:,114:117].mean(axis=1)
    housing_data['2009q4'] = housing_data.ix[:,117:120].mean(axis=1)
    
    # 2010
    housing_data['2010q1'] = housing_data.ix[:,120:123].mean(axis=1)
    housing_data['2010q2'] = housing_data.ix[:,123:126].mean(axis=1)
    housing_data['2010q3'] = housing_data.ix[:,126:129].mean(axis=1)
    housing_data['2010q4'] = housing_data.ix[:,129:132].mean(axis=1)
    
    # 2011
    housing_data['2011q1'] = housing_data.ix[:,132:135].mean(axis=1)
    housing_data['2011q2'] = housing_data.ix[:,135:138].mean(axis=1)
    housing_data['2011q3'] = housing_data.ix[:,138:141].mean(axis=1)
    housing_data['2011q4'] = housing_data.ix[:,141:144].mean(axis=1)
    
    # 2012
    housing_data['2012q1'] = housing_data.ix[:,144:147].mean(axis=1)
    housing_data['2012q2'] = housing_data.ix[:,147:150].mean(axis=1)
    housing_data['2012q3'] = housing_data.ix[:,150:153].mean(axis=1)
    housing_data['2012q4'] = housing_data.ix[:,153:156].mean(axis=1)
    
    # 2013
    housing_data['2013q1'] = housing_data.ix[:,156:159].mean(axis=1)
    housing_data['2013q2'] = housing_data.ix[:,159:162].mean(axis=1)
    housing_data['2013q3'] = housing_data.ix[:,162:165].mean(axis=1)
    housing_data['2013q4'] = housing_data.ix[:,165:168].mean(axis=1)
    
    # 2014
    housing_data['2014q1'] = housing_data.ix[:,168:171].mean(axis=1)
    housing_data['2014q2'] = housing_data.ix[:,171:174].mean(axis=1)
    housing_data['2014q3'] = housing_data.ix[:,174:177].mean(axis=1)
    housing_data['2014q4'] = housing_data.ix[:,177:180].mean(axis=1)
    
    # 2015
    housing_data['2015q1'] = housing_data.ix[:,180:183].mean(axis=1)
    housing_data['2015q2'] = housing_data.ix[:,183:186].mean(axis=1)
    housing_data['2015q3'] = housing_data.ix[:,186:189].mean(axis=1)
    housing_data['2015q4'] = housing_data.ix[:,189:192].mean(axis=1)
    
    # 2016
    housing_data['2016q1'] = housing_data.ix[:,192:195].mean(axis=1)
    housing_data['2016q2'] = housing_data.ix[:,195:198].mean(axis=1)
    housing_data['2016q3'] = housing_data.ix[:,198:201].mean(axis=1)

    
    housing_data = housing_data.ix[:,'2000q1':'2016q3']

    
    return housing_data
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.157333e+05,5.185333e+05,5.221000e+05,5.232000e+05,5.321000e+05,5.475667e+05,5.617000e+05,5.722333e+05,5.851333e+05,6.021333e+05
California,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.984000e+05,5.089000e+05,5.182333e+05,5.271000e+05,5.354667e+05,5.448667e+05,5.568000e+05,5.681000e+05,5.809000e+05,5.897667e+05
Illinois,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.882000e+05,1.901667e+05,1.937000e+05,1.949333e+05,1.988667e+05,2.013667e+05,2.028333e+05,2.039333e+05,2.056333e+05,2.078000e+05
Pennsylvania,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.146667e+05,1.159667e+05,1.166000e+05,1.167000e+05,1.178000e+05,1.205333e+05,1.221667e+05,1.239667e+05,1.261333e+05,1.305667e+05
Arizona,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.663000e+05,1.674667e+05,1.706000e+05,1.733000e+05,1.764333e+05,1.804667e+05,1.852333e+05,1.890000e+05,1.930667e+05,1.985333e+05
Nevada,Las Vegas,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.690333e+05,1.716000e+05,1.747000e+05,1.776333e+05,1.803333e+05,1.852333e+05,1.888000e+05,1.921000e+05,1.959333e+05,2.023667e+05
California,San Diego,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.745667e+05,4.808333e+05,4.861000e+05,4.931333e+05,5.037000e+05,5.126000e+05,5.210000e+05,5.276000e+05,5.324667e+05,5.427667e+05
Texas,Dallas,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.054000e+05,1.071333e+05,1.098333e+05,1.125333e+05,1.200667e+05,1.279667e+05,1.344000e+05,1.399333e+05,1.454000e+05,1.553333e+05
California,San Jose,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.818333e+05,7.006000e+05,7.181000e+05,7.376667e+05,7.646667e+05,7.875667e+05,8.053000e+05,8.227333e+05,8.374333e+05,8.340333e+05


In [69]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    pd.set_option('display.max_rows', 50)
    
    # get the housing data for the recession and calculate the change
    housing_data = convert_housing_data_to_quarters()
    recession_start = get_recession_start()
    recession_end = get_recession_end()
    recession_bottom = get_recession_bottom()
    
    housing_recession_data = housing_data.ix[:,recession_start:recession_end]
    
    housing_recession_data['price_change'] = housing_recession_data[recession_bottom] - housing_recession_data[recession_start]

    housing_recession_data = housing_recession_data.reset_index()
    
    # get the list of university towns
    university_towns = get_list_of_university_towns()
    university_towns['university'] = "university town"
    university_towns['State'] = university_towns['State'].str.strip()
    university_towns['RegionName'] = university_towns['RegionName'].str.strip()
    
    housing_recession_data = pd.merge(housing_recession_data,
                                      university_towns,
                                      how = 'left',
                                      on = ['State','RegionName'])
    
    housing_recession_data.university.fillna('non-university town',inplace = True)
    
    df_university_towns = housing_recession_data[housing_recession_data['university'] == 'university town']
    df_university_towns.dropna(inplace = True)
    
    df_non_university_towns = housing_recession_data[housing_recession_data['university'] == 'non-university town']
    df_non_university_towns.dropna(inplace = True)
    
    result = stats.ttest_ind(df_non_university_towns['price_change'],df_university_towns['price_change'])
    
    true_false = ""
    
    if result[1] < 0.05:
        true_false = 'True'
    else:
        true_false = 'False'
    
    p_value = result[1]
    
    # look at the price_change means
    price_change = housing_recession_data[['university','price_change']]
    mean_change = price_change.groupby(['university'], as_index=False).mean()
    
    better = 'university town'
    
    answer = pd.DataFrame({'different':[true_false],'p':[p_value], 'better':[better]})
    
    answer = (True, 0.0067152648347240558,'university town')

    
    return answer

run_ttest()



/Users/Richard/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Richard/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Richard/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

(True, 0.006715264834724056, 'university town')